In [1]:
# importing packages
from pytube import YouTube, Playlist, extract, helpers
import os
import json

In [2]:
# https://www.geeksforgeeks.org/download-video-in-mp3-format-using-pytube/
def youtube_url_to_mp3(yt_url, playlist_path="mp3/"):
    # url input from user
    yt = YouTube(yt_url)
    # extract only audio
    video = yt.streams.filter(only_audio=True).first()

    destination = playlist_path + '/'
    
    # download the file
    out_file = video.download(output_path=destination)
    
    # save the file
    base, ext = os.path.splitext(out_file) 
    filename = destination + extract.video_id(yt_url) + '.mp3'
    os.rename(out_file, filename) 

    # # save the file
    # base, ext = os.path.splitext(out_file)
    # new_file = base + '.mp3'
    # os.rename(out_file, new_file)

    print('\n created MP3 for' + yt_url + ' at ' + filename)

    # print("Downloaded successfully: \n", new_file)
    return yt, filename 


In [3]:
#karpathy_url = "https://www.youtube.com/watch?v=VMj-3S1tku0&list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ&index=1&t=112s"
karpathy_url = "https://www.youtube.com/watch?v=caWEOyNsU3Y"
karpathy_obj = youtube_url_to_mp3(karpathy_url)
print (karpathy_obj)

# fastai_url = "https://www.youtube.com/watch?v=F4tvM4Vb3A0&list=PLfYUBJiXbdtSvpQjSnJJ_PmDQB_VyT5iU"
# fastai_obj = youtube_url_to_mp3(fastai_url)
# print (fastai_obj)



 created MP3 forhttps://www.youtube.com/watch?v=caWEOyNsU3Y at mp3//caWEOyNsU3Y.mp3
(<pytube.__main__.YouTube object: videoId=caWEOyNsU3Y>, 'mp3//caWEOyNsU3Y.mp3')


In [4]:
def get_urls_from_youtube_playlist(playlist_url):
    """Returns a list of video urls from a youtube playlist"""
    playlist = Playlist(playlist_url)
    return playlist.video_urls

In [5]:
karpathy_playlist = "https://www.youtube.com/playlist?list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ"

karpathy_urls = get_urls_from_youtube_playlist(karpathy_playlist)
karpathy_playlist_id = extract.playlist_id(karpathy_playlist)

karpathy_playlist_vid_ids = [extract.video_id(url) for url in karpathy_urls]

karpathy_playlist_vid_ids

['VMj-3S1tku0', 'PaCmpygFfXo', 'TCH_1BHY58I', 'P6sfmUTpUmc', 'q8SA3rM6ckI']

In [6]:
# Andrej Karpathy 
karpathy_playlist_url = "https://www.youtube.com/playlist?list=PL9dX7Elz2t0-c4Sy7trowtrDlbRWaoi6f"

karpathy_playlist_shorts_url = "https://www.youtube.com/playlist?list=PL9zq2zalZB1ID7wBz5fi9cQ8jZB3FH07q"

# Fei Fei Li
feifei_playlist_url = "https://youtube.com/playlist?list=PL9dX7Elz2t08Az4p5MghL1EpfQXs9urPx"

# Yann LeCun
lecun_playlist_url = "https://youtube.com/playlist?list=PL9dX7Elz2t09sz_rNmRDPCux-ZGC09vWS"

In [7]:
!pip3 install importlib-metadata

from importlib.metadata import metadata
from operator import length_hint

ModuleNotFoundError: No module named 'importlib.metadata'

In [8]:
def create_yt_metadata(yt, url,person, whisper_result=''):

    vid_json_metadata = {
        "current_person": person,
        "video_id": extract.video_id(url),
        "url": url,
        "title": yt.title,
        "description": yt.description,
        "author": yt.author,
        "keywords": yt.keywords,
        "channel_url": yt.channel_url,
        "length": yt.length,
        "views": yt.views,
        "whisper_result": whisper_result,
    }

    return vid_json_metadata


In [9]:
## DEFINE CUSTOM VARIABLES FOR THIS PLAYLIST 

# playlist_url = "https://www.youtube.com/playlist?list=PL9zq2zalZB1ID7wBz5fi9cQ8jZB3FH07q" #karpathy shorts 

playlist_url = feifei_playlist_url
playlist_path = "feifeili"
current_person = "Fei-Fei Li"

# ============================================

# # loop through a playlist 
video_metadata = {}

# ============================================
# THE MAIN LOOP 
# ============================================

vid_urls = get_urls_from_youtube_playlist(playlist_url)

# get all the videos 
for url in vid_urls:

    try: 
    
        ## =======
        ## Save the Video as an MP3
        yt, path = youtube_url_to_mp3(url, playlist_path)
        print ('yt', yt)
        print ('path', path)

        # transcribe the resulting mp3 file
        # try: 
        #     whisper_result = model.transcribe(path)
        #     print ('created whisper result for ' + path)
        # except Exception as e: 
        #     print ('could not create whisper result for ' + path)
        #     print (e)
        #     whisper_result = ''

        # save a json object for the whisper_result

        # create the json object and then save it 
        yt_video_id = extract.video_id(url)
        vid_json_metadata = create_yt_metadata(yt, url, current_person, whisper_result='')
        video_metadata[url] = vid_json_metadata

        print ('created json metadata for ' + yt.title)

    except Exception as e:
        print (e)
    

# create a json object from video_metadata
# json_data = json.dumps(video_metadata)

with open(playlist_path + '__video_metadata.json', 'w') as f:
    json.dump(video_metadata, f, indent=2)


 created MP3 forhttps://www.youtube.com/watch?v=_JuQcodHANs at karpathy_shorts/_JuQcodHANs.mp3
yt <pytube.__main__.YouTube object: videoId=_JuQcodHANs>
path karpathy_shorts/_JuQcodHANs.mp3
created json metadata for Andrej Karpathy: Setting ambitious goals at Tesla | Lex Fridman Podcast Clips

 created MP3 forhttps://www.youtube.com/watch?v=v5l-jPsAK7k at karpathy_shorts/v5l-jPsAK7k.mp3
yt <pytube.__main__.YouTube object: videoId=v5l-jPsAK7k>
path karpathy_shorts/v5l-jPsAK7k.mp3
created json metadata for Path prediction (Andrej Karpathy)

 created MP3 forhttps://www.youtube.com/watch?v=x0_404KeDIc at karpathy_shorts/x0_404KeDIc.mp3
yt <pytube.__main__.YouTube object: videoId=x0_404KeDIc>
path karpathy_shorts/x0_404KeDIc.mp3
created json metadata for Depth From Vision: Self-Supervision (Andrej Karpathy)


In [10]:
def make_yt_url_from_video_id(video_id):
    return "https://www.youtube.com/watch?v=" + video_id
    

In [12]:
# open the json file

merged_folder_path = 'final_data' 

print (playlist_path)

with open(playlist_path + '__video_metadata.json') as f:
    data = json.load(f)
    #edit the "whisper_object" key in the json file
    for url in data:
        video_id = data[url]['video_id']
        # get the whisper object from the video id
        # load the whiser_object json file 
        with open('whisper_objects/whisper_objects__' + playlist_path + '.json') as f:
            whisper_data = json.load(f)

            # whisper_object = get_whisper_object_from_video_id(video_id)
            # print ('f text \n', f.read())

            data[url]['whisper_result'] = whisper_data[url]
    # save the json file
    with open(str(merged_folder_path +'/'+playlist_path + '__video_metadata_MERGED.json'), 'w') as f:
        json.dump(data, f, indent=2)



karpathy_shorts


In [ ]:
# print(data)